# Related Series
* Constructs a graph of structurally related series (sequels, prequels, etc.)

In [ ]:
medium = ""

In [ ]:
const name = "$medium/all/CrossRelatedSeries";

In [ ]:
using DataFrames
using LinearAlgebra
using SparseArrays
import CSV
import DataFrames: DataFrame
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");

## Compute similarity matrices

In [ ]:
function get_source_media(medium)
    if medium == "anime"
        source_media = "manga"
    elseif medium == "manga"
        source_media = "anime"
    else
        @assert false
    end
end;

In [ ]:
function get_relations(medium)
    source_media = get_source_media(medium)
    file = get_data_path("processed_data/$(source_media)_$(medium)_related_series.csv")
    df = DataFrame(CSV.File(file, stringtype = String))
    replace!(
        df.relationship,
        "parent" => "parent_story",
        "alternative" => "alternative_version",
        "source" => "adaptation",
    )
    df
end;

In [ ]:
function get_similarity_matrix(medium, relationships)
    df = get_relations(medium)
    df = filter(x -> x.relationship ∈ relationships, df)
    sparse(
        df.source .+ 1,
        df.target .+ 1,
        ones(Float32, length(df.source)),
        num_items(get_source_media(medium)),
        num_items(medium),
    )
end;

In [ ]:
# definitions for the relationships can be found at https://myanimelist.net/info.php?go=relationinfo
const strict_relations = Set(("sequel", "prequel", "parent_story", "side_story"))
const recap_relations = Set(("alternative_version", "summary", "full_story", "adaptation"))
const loose_relations = Set(("alternative_setting", "spin_off"))
const no_relations = Set(("character", "other"))
const all_relations =
    union(strict_relations, recap_relations, loose_relations, no_relations)
@assert all(map(x -> x ∈ all_relations, get_relations(medium).relationship))

In [ ]:
S = get_similarity_matrix(medium, all_relations)
write_params(Dict("S" => S), "$medium/all/CrossRelatedSeries")

In [ ]:
S = get_similarity_matrix(medium, recap_relations)
write_params(Dict("S" => S), "$medium/all/CrossRecapSeries")